In [36]:
import hashlib
import random
# Thư viện random được sử dụng cho các khóa phiên, nhưng để đồng nhất với bản word em đã cố định các khóa phiên ở code bên dưới

In [37]:
def hashing(message):
    hash_object = hashlib.sha256(message.encode())
    return int.from_bytes(hash_object.digest(), 'big')

In [38]:
class EllipticCurve:
    def __init__(self, a, b, p):
        self.a = a
        self.b = b
        self.p = p
        if not self.check_condition():
            raise ValueError("Đường cong không hợp lệ")
        
    def check_condition(self):
        return (4 * self.a**3 + 27 * self.b**2) % self.p != 0

    def is_point_on_curve(self, point):
        x, y = point
        return (y**2) % self.p == (x**3 + self.a * x + self.b) % self.p

    def add_points(self, p1, p2):
        x1, y1 = p1
        x2, y2 = p2
        if p1 == (0, 0):
            return p2
        if p2 == (0, 0):
            return p1
        if x1 == x2 and y1 == -y2 % self.p:
            return (0, 0)
        if x1 == x2 and y1 == y2:
            m = (3 * x1**2 + self.a) * pow(2 * y1, -1, self.p) % self.p
        else:
            m = (y2 - y1) * pow((x2 - x1) % self.p, -1, self.p) % self.p
        x3 = (m**2 - x1 - x2) % self.p
        y3 = (m * (x1 - x3) - y1) % self.p
        return (x3, y3)

    def multiply_point(self, point, n):
        if n == 0:
            return (0, 0)  
        if n == 1:
            return point
        result = (0, 0)  
        addend = point

        while n > 0:
            if n % 2 == 1:  # Nếu n là lẻ
                result = self.add_points(result, addend)
            addend = self.add_points(addend, addend)
            n //= 2

        return result


In [39]:
class EllipticCurveCryptography():
    def __init__(self, curve, G, n):
        self.curve = curve
        self.G = G
        self.n = n
        self.s = 2230918470331275586959590240276578437692550425728795580234906002400542218242875896943023205186936869621443444560558819650762211369742687479307925220760881578
        # self.s = random.randint(1, n - 1)
        self.B = self.curve.multiply_point(self.G, self.s)
        
    def get_public_key(self):
        return (self.curve, self.n, self.G, self.B)
    
    def get_private_key(self):
        return self.s

    def encrypt(self, M):
        #k = random.randint(1, self.n - 1)
        k = 4294185155803377833565689081824777287592567687422837919611491881507290245705633945565224761202575939371316105223800412671669321306799804552963165055774976709
        M1 = self.curve.multiply_point(self.G, k)
        M2 = self.curve.add_points(M, self.curve.multiply_point(self.B, k))
        return (M1, M2)

    def decrypt(self, C1, C2):
        return self.curve.add_points(C2, self.curve.multiply_point(C1, -self.s))

In [40]:
class Participant():
    def __init__(self, curve, G, n, M, message):
        self.ECC = EllipticCurveCryptography(curve, G, n)
        self.M = M
        self.message = message

    def encrypt_message(self):
        print("Mã hóa thông điệp:")
        return self.ECC.encrypt(self.M)

    def decrypt_message(self, C1, C2):
        print("Giải mã thông điệp:")
        return self.ECC.decrypt(C1, C2)
    
    def sign_message(self):
        n, G, B = self.ECC.get_public_key()[1:]
        s = self.ECC.get_private_key()
        
        while True:
            # k = random.randint(1, n - 1)
            k = 4294185155803377833565689081824777287592567687422837919611491881507290245705633945565224761202575939371316105223800412671669321306799804552963165055774976709
            M1 = self.ECC.curve.multiply_point(G, k)
            r = M1[0] % n
            if r == 0:
                continue
            h = hashing(self.message)
            u = (h + s * r) * pow(k, -1, n) % n
            if u == 0:
                continue
            return (r, u)

    def verify_signature(self, signature):
        print("Xác minh chữ ký:")
        r, u = signature
        print(f"Chữ ký (r, u): ({r}, {u})")
        
        n, G, B = self.ECC.get_public_key()[1:]
        
        w = pow(u, -1, n)
        print(f"Bước 1: Tính w: w = u^(-1) mod n = {u}^(-1) mod {n} = {w}")
        
        # Bước 2: Tính h
        h = hashing(self.message)
        print(f"Bước 2: Tính h(M): h = hashing(message) = {hashing(self.message)}")

        # Bước 3: Tính u1 và u2
        u1 = (h * w) % n
        u2 = (r * w) % n
        print(f"Bước 3: Tính u1 và u2: u1 = (h * w) mod n = ({h} * {w}) mod {n} = {u1}")
        print(f"u2 = (r * w) mod n = ({r} * {w}) mod {n} = {u2}")

        # Bước 4: Tính P
        P = self.ECC.curve.add_points(
            self.ECC.curve.multiply_point(G, u1),
            self.ECC.curve.multiply_point(B, u2)
        )
        print(f"Bước 4: Tính P: P = G * u1 + B * u2 = {P}")

        # Kiểm tra điều kiện
        valid = r == P[0] % n
        print(f"Bước 5: Kết quả kiểm tra: r == P[0] mod n: {r} == {P[0] % n} => {valid}")
        
        return valid

In [41]:
# Thiết lập tham số cho đường cong elliptic
p = 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151
n = 6864797660130609714981900799081393217269435300143305409394463459185543183397655394245057746333217197532963996371363321113864768612440380340372808892707005449
a = -3
b = 1093849038073734274511112390766805569936207598951683748994586394495953116150735016013708737573759623248592132296706313309438452531591012912142327488478985984
G = (2661740802050217063228768716723360960729859168756973147706671368418802944996427808491545080627771902352094241225065558662157113545570916814161637315895999846,
    3757180025770020463545507224491183603594455134769762486694567779615544477440556316691234405012945539562144444537289428522585666729196580810124344277578376784)
M = (
    550033349500675809210630911932614746689308056783116555503796076441997314296552229414596056289308795809230495161107375944280740062460523807998150903814049756,
    6314764310629933905771269887148778470580127243360188853890667382743545869101103822707963584372145759168065816230373482092841247937256120004575877387301007395
)
message = "Hoang Kim Chi"

In [42]:
participant1 = Participant(EllipticCurve(a, b, p), G, n, M, message)

# Hiển thị khóa công khai và khóa bí mật
print("Khóa công khai:", participant1.ECC.get_public_key()[1:])
print("Khóa bí mật:", participant1.ECC.get_private_key())

# Mã hóa và giải mã thông điệp
C1, C2 = participant1.encrypt_message()
print("Bản mã (C1, C2):", (C1, C2))
decrypted_message = participant1.decrypt_message(C1, C2)
print("Bản rõ sau khi giải mã:", decrypted_message)

# Ký và xác minh chữ ký
signature = participant1.sign_message()
print("Chữ ký (r, u):", signature)
is_valid_signature = participant1.verify_signature(signature)
print("Chữ ký hợp lệ:", is_valid_signature)

Khóa công khai: (6864797660130609714981900799081393217269435300143305409394463459185543183397655394245057746333217197532963996371363321113864768612440380340372808892707005449, (2661740802050217063228768716723360960729859168756973147706671368418802944996427808491545080627771902352094241225065558662157113545570916814161637315895999846, 3757180025770020463545507224491183603594455134769762486694567779615544477440556316691234405012945539562144444537289428522585666729196580810124344277578376784), (139917486449223876417376999887501749479009497671265011115828546855414808150791469851810414103311347061717943396395355660104772267774606436057201810523366654, 3791872272530666163238392930784735258501918984120147950109150993999986927884447348820467582071330841119942743109908623256552932818296339100188698424193216026))
Khóa bí mật: 2230918470331275586959590240276578437692550425728795580234906002400542218242875896943023205186936869621443444560558819650762211369742687479307925220760881578
Mã hóa thông đ